In [11]:
import requests
import os
import json
import tweepy
from config import BEARER_TOKEN

In [12]:
# export 'BEARER_TOKEN'='<your_bearer_token>'
os.environ['bearer_token'] = BEARER_TOKEN
bearer_token = os.environ.get('bearer_token')

In [13]:
search_url = "https://api.twitter.com/1.1/search/tweets.json?q=%23provaccine OR antivaccine OR covidvaccine&result_type=recent"

In [14]:
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}


In [15]:

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r


In [16]:
class MyStreamListener(tweepy.StreamListener):
    
    def __init__(self, time_limit=300):
        self.start_time = time.time()
        self.limit = time_limit
        super(MyStreamListener, self).__init__()
    
    def on_connect(self):
        print("Connected to Twitter API.")
        
    def on_status(self, status):
        
        
        # Tweet ID
        tweet_id = status.id
        
        # User ID
        user_id = status.user.id
        # Username
        username = status.user.name
        
        
        # Tweet
        if status.truncated == True:
            tweet = status.extended_tweet['full_text']
            hashtags = status.extended_tweet['entities']['hashtags']
        else:
            tweet = status.text
            hashtags = status.entities['hashtags']
        
        # Read hastags
        hashtags = read_hashtags(hashtags)            
        
        # Retweet count
        retweet_count = status.retweet_count
        # Language
        lang = status.lang
        # Geograpic
        geo = status.geo_coordinates
    # If tweet is not a retweet and tweet is in English
        if not hasattr(status, "retweeted_status") and lang=="en":
            # Connect to database
            dbConnect(user_id, username, tweet_id, tweet, retweet_count, hashtags)
            
        if (time.time() - self.start_time) > self.limit:
            
            print(time.time(), self.start_time, self.limit)
            return False
            
    def on_error(self, status_code):
        if status_code == 420:
            # Returning False in on_data disconnects the stream
            return False

In [17]:
def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


In [18]:
def main():
    json_response = connect_to_endpoint(search_url, query_params)
    print(json.dumps(json_response, indent=4, sort_keys=True))


In [19]:
if __name__ == "__main__":
    tweet_json = main()
    print(tweet_json)

200
{
    "search_metadata": {
        "completed_in": 0.081,
        "count": 15,
        "max_id": 1431714818566893577,
        "max_id_str": "1431714818566893577",
        "next_results": "?max_id=1431713357283823617&q=%23provaccine%20OR%20antivaccine%20OR%20covidvaccine&include_entities=1&result_type=recent",
        "query": "%23provaccine+OR+antivaccine+OR+covidvaccine",
        "refresh_url": "?since_id=1431714818566893577&q=%23provaccine%20OR%20antivaccine%20OR%20covidvaccine&result_type=recent&include_entities=1",
        "since_id": 0,
        "since_id_str": "0"
    },
    "statuses": [
        {
            "contributors": null,
            "coordinates": null,
            "created_at": "Sat Aug 28 20:26:27 +0000 2021",
            "entities": {
                "hashtags": [
                    {
                        "indices": [
                            21,
                            30
                        ],
                        "text": "BREAKING"
          

AttributeError: 'NoneType' object has no attribute 'read'